# Part 1: Connectiong to the database

Import [Postgres.py](https://postgres-py.readthedocs.io/en/latest/#), a high level PostgreSQL client library, which is used for most interactions with the database in this tutorial (except in Example 2d).

In [ ]:
import postgres

Connect to PostgreSGL database.

In [ ]:
# Connection parameters.
pg_user = 'postgres'
pg_password = 'postgres'
pg_host = 'localhost'
pg_port = '5432'
db_name = 'integrcity_workshop'

# Construct connection string.
db_connection_string = 'postgres://{0}:{1}@{2}:{3}/{4}'.format( pg_user, pg_password, pg_host, pg_port, db_name )

# Make connection
db = postgres.Postgres( db_connection_string )

# Part 2: Retrieving data from the database

#### Example 2a: Retrieve IDs of city object groups

A simple query: Retrieve IDs of heating supply areas.

In [ ]:
sql_command = open( 'sql/retrieve_heating_supply_areas.sql' ).read()

id_space_heating_supply_area1 = db.one( sql_command, { 'func' : 'space_heating_supply_area_1' } )
id_space_heating_supply_area2 = db.one( sql_command, { 'func' : 'space_heating_supply_area_2' } )


print 'space heating supply areas:'
print '\t area 1: id =', id_space_heating_supply_area1
print '\t area 2: id =', id_space_heating_supply_area2

#### Example 2b: Retrieve IDs of buildings

A query using the **INNER JOIN** statement: Retrieve IDs of buildings (object class ID = 26) that are associated with the heating supply areas.

In [ ]:
sql_command = open( 'sql/retrieve_building_ids.sql' ).read()

building_ids_supply_area1 = db.all( sql_command, { 'ogid' : id_space_heating_supply_area1 } )
building_ids_supply_area2 = db.all( sql_command, { 'ogid' : id_space_heating_supply_area2 } )

print 'buildings in supply areas:'
print '\t area 1: ids =', building_ids_supply_area1
print '\t area 2: ids =', building_ids_supply_area2

#### Example 2c: Retrieve data for buildings

A query using multiple **INNER JOIN** statements: Retrieve time series info (type 'RegularTimeSeries', object class ID = 202) for energy demand (object class ID = 232) for space heating (end_use = SpaceHeating) associated with these buildings.

In [ ]:
sql_command = open( 'sql/retrieve_building_data.sql' ).read()

building_data_supply_area1 = []
for bid in building_ids_supply_area1:
    data = db.one( sql_command, { 'coid' : bid } )
    building_data_supply_area1.append( data )

building_data_supply_area2 = []
for bid in building_ids_supply_area2:
    data = db.one( sql_command, { 'coid' : bid } )
    building_data_supply_area2.append( data )

print 'builing data for supply area 1:'
for data in building_data_supply_area1: print '\t building id', data.id, ':', data.type, 'from %i' % data.year_of_construction
print 'builing data for supply area 2:'
for data in building_data_supply_area2: print '\t building id', data.id, ':', data.type, 'from %i' % data.year_of_construction

#### Example 2d: Retrieve IDs of buildings using SQLAlchemy

[SQLAlchemy](http://www.sqlalchemy.org/) is designed for efficient and high-performing database access, adapted into a simple domain language. Among other things, it provides a system of constructing and representing SQL commands using Python constructs.

First, connect to the database and retrieve handles to tables.

In [ ]:
import sqlalchemy
import warnings

# Connect to the database
engine = sqlalchemy.create_engine( db_connection_string )

# Retrieve meta data (including information about existing tables).
meta = sqlalchemy.MetaData()
with warnings.catch_warnings():
    warnings.simplefilter( 'ignore', category = sqlalchemy.exc.SAWarning )
    meta.reflect( bind = engine )

# Retrieve handles to tables.
cityobject = meta.tables[ 'cityobject' ]
group_to_cityobject = meta.tables[ 'group_to_cityobject' ]

Construct an SQL command directly in Python for retrieving the IDs of buildings (compare Example 2b). This ensures that the command is syntactically correct and corresponds to the data available in the database.

In [ ]:
# Set aliases (just for convenience).
co = cityobject.alias()
gm = group_to_cityobject.alias()

# Construct SQL command using SQLAlchemy.
sql_command = \
    sqlalchemy.sql.select( 
        [ gm.c.cityobject_id.label( 'id' ) ]
    ).select_from(
        gm.join( co, gm.c.cityobject_id == co.c.id )
    ).where(
        sqlalchemy.sql.and_(
            gm.c.cityobjectgroup_id == sqlalchemy.sql.bindparam( 'cog' ),
            co.c.objectclass_id == 26,
        )
    ).order_by( 'id' )

print sql_command

Retrieve the information from the database.

In [ ]:
# Execute query.
connection = engine.connect()
building_ids_supply_area1_sqlalchemy = connection.execute( sql_command, cog = id_space_heating_supply_area1 )

print 'buildings in supply area 1:'
print [ r for r in building_ids_supply_area1_sqlalchemy ]

# Result should be explicitly closed before discarding.
building_ids_supply_area1_sqlalchemy.close()

# Part 3: Working with data

#### Example 3a: Plotting time series for different types of buildings

Define a figure for plotting the time series.

In [ ]:
import matplotlib.pyplot as plotter

hours_per_day = 24
hours_of_day = range( 1, hours_per_day + 1 )

# Define new figure.
figure, axes = plotter.subplots( nrows=2, ncols=3 )

# Subplot: apartment blocks in supply area 1
axes[0,0].set_title( 'area 1:\napartment blocks', fontsize=9 )
axes[0,0].set_xlabel( 'time in h', fontsize=8 )
axes[0,0].set_ylabel( 'demand in kWh', fontsize=8 )
axes[0,0].set_xlim( [ 0.5 ,24.5 ] )

# Subplot: multi-family houses in supply area 1
axes[0,1].set_title( 'area 1:\nmulti-family homes', fontsize=9 )
axes[0,1].set_xlabel( 'time in h', fontsize=8 )
axes[0,1].set_ylabel( 'demand in kWh', fontsize=8 )
axes[0,1].set_xlim( [ 0.5 ,24.5 ] )

# Subplot: single-family houses in supply area 1
axes[0,2].set_title( 'area 1:\nsingle-family homes', fontsize=9 )
axes[0,2].set_xlabel( 'time in h', fontsize=8 )
axes[0,2].set_ylabel( 'demand in kWh', fontsize=8 )
axes[0,2].set_xlim( [ 0.5 ,24.5 ] )

# Subplot: apartment blocks in supply area 2
axes[1,0].set_title( 'area 2:\napartment blocks', fontsize=9 )
axes[1,0].set_xlabel( 'time in h', fontsize=8 )
axes[1,0].set_ylabel( 'demand in kWh', fontsize=8 )
axes[1,0].set_xlim( [ 0.5 ,24.5 ] )

# Subplot: multi-family houses in supply area 2
axes[1,1].set_title( 'area 2:\nmulti-family house', fontsize=9 )
axes[1,1].set_xlabel( 'time in h', fontsize=8 )
axes[1,1].set_ylabel( 'demand in kWh', fontsize=8 )
axes[1,1].set_xlim( [ 0.5 ,24.5 ] )

# Subplot: single-family houses in supply area 2
axes[1,2].set_title( 'area 2:\nsingle-family house', fontsize=9 )
axes[1,2].set_xlabel( 'time in h', fontsize=8 )
axes[1,2].set_ylabel( 'demand in kWh', fontsize=8 )
axes[1,2].set_xlim( [ 0.5 ,24.5 ] )

# Define size of figure.
figure.set_size_inches( 10, 4 )

Plot the time series.

In [ ]:
# Fill building data from supply area 1.
for data in building_data_supply_area1:
    if data.type == 'Apartment Block':
        axes[0,0].plot( hours_of_day, data.values_array, 'ro-' )
    elif data.type == 'Multi-Family House':
        axes[0,1].plot( hours_of_day, data.values_array, 'rv-' )
    elif data.type == 'Single-Family House':
        axes[0,2].plot( hours_of_day, data.values_array, 'r^-' )

# Fill building data from supply area 2.
for data in building_data_supply_area2:
    if data.type == 'Apartment Block':
        axes[1,0].plot( hours_of_day, data.values_array, 'bo-' )
    elif data.type == 'Multi-Family House':
        axes[1,1].plot( hours_of_day, data.values_array, 'bv-' )
    elif data.type == 'Single-Family House':
        axes[1,2].plot( hours_of_day, data.values_array, 'b^-' )

# Plot the figure.
figure.tight_layout()
plotter.show()

#### Example 3b: Aggregating time series for supply areas

Compute an aggregated demand profile for each supply area.

In [ ]:
from decimal import *

def aggregate_demand_profiles( data, size ):
    result = []
    for i in range( 0, size ):
        result.append( 0 )
        for d in data: result[i] += d.values_array[i]
    return result

aggregated_profiles_area1 = aggregate_demand_profiles( building_data_supply_area1, hours_per_day )
aggregated_profiles_area2 = aggregate_demand_profiles( building_data_supply_area2, hours_per_day )

#### Example 3c: Aggregate time series weighted by year of construction

Assume a scenario where space heating demand is reduced by 50% for buildings built in 1985 or before due to refurbishment. For this case, weights for the demand profiles based on the building's year of contruction can be used.

In [ ]:
def weighted_aggregate_demand_profiles( data, size ):
    result = []
    for i in range( 0, size ):
        result.append( 0 )
        for d in data:
            weight = Decimal(1.0) if ( d.year_of_construction > 1985 ) else Decimal(0.5)
            result[i] += d.values_array[i] * weight
    return result

weighted_aggregated_profiles_area1 = weighted_aggregate_demand_profiles( building_data_supply_area1, hours_per_day )
weighted_aggregated_profiles_area2 = weighted_aggregate_demand_profiles( building_data_supply_area2, hours_per_day )

#### Example 3d: Plot the aggregared demand profiles for comparision

Create a new figure and plot the time series.

In [ ]:
# Define new figure.
figure, axes = plotter.subplots( nrows=1, ncols=2, sharey=True )

axes[0].set_title( 'energy demand: status quo', fontsize=10 )
axes[1].set_title( 'energy demand: refurbished', fontsize=10 )

axes[0].set_xlabel( 'time in h', fontsize=9 )
axes[1].set_xlabel( 'time in h', fontsize=9 )

axes[0].set_ylabel( 'thermal demand in kWh', fontsize=9 )

# Plot time series
axes[0].plot( hours_of_day, aggregated_profiles_area1, 'ro-' )
axes[0].plot( hours_of_day, aggregated_profiles_area2, 'bd-' )

axes[1].plot( hours_of_day, weighted_aggregated_profiles_area1, 'ro-' )
axes[1].plot( hours_of_day, weighted_aggregated_profiles_area2, 'bd-' )

# Plot the figure.
figure.tight_layout()
plotter.show()

# Part 4: Updating the database

Define colors (RGB scheme) for visualizing results.

In [ ]:
def get_rgb_color_string( val, cmap_name = 'coolwarm' ):
    color = plotter.get_cmap( cmap_name )( float( val ) )
    return '{:4.3f} {:4.3f} {:4.3f}'.format( color[0], color[1], color[2] )

## Define color of buildings in each supply area according to aggregated peak demand.
#rgb_color_area1 = get_rgb_color_string( max(aggregated_profiles_area1) / 500 )
#rgb_color_area2 = get_rgb_color_string( max(aggregated_profiles_area2) / 500 )

## Define color of buildings in each supply area according to (weighted) aggregated peak demand.
#rgb_color_area1 = get_rgb_color_string( max(weighted_aggregated_profiles_area1) / 500 )
#rgb_color_area2 = get_rgb_color_string( max(weighted_aggregated_profiles_area2) / 500 )

## Use yellow for buildings in supply area 1 and cyan for buildings in supply area 2.
rgb_color_area1 = '1.0 1.0 0.0'
rgb_color_area2 = '0.0 1.0 1.0'

Update the database for visualization. More specifically, change the color of the sufaces called *displayResultsAttributes* associated with the appearance/theme titled *displayResults*.

This information may then be exported as KML/COLLADA/glTF model using the [3DCityDB Importer/Exporter](http://www.3dcitydb.org/3dcitydb/3dimpexp/) and viewed in [Google Earth Pro](https://www.google.de/earth/download/gep/agree.html).

In [ ]:
sql_command = open( 'sql/update_appearance_supply_area.sql' ).read()

db.run( sql_command, { 'cog' : 'space_heating_supply_area_1', 'rgb' : rgb_color_area1 } )
db.run( sql_command, { 'cog' : 'space_heating_supply_area_2', 'rgb' : rgb_color_area2 } )